# Train

In [1]:
from __future__ import print_function
import six
import sys
import os
from optparse import OptionParser
from keras.models import load_model, Model, model_from_json
from argparse import ArgumentParser
from keras import backend as K
from keras.layers import Input
import numpy as np
# fix random seed for reproducibility
seed = 39
np.random.seed(seed)
import h5py
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pandas as pd
from keras.optimizers import Adam, Nadam
from callbacks import all_callbacks
import pandas as pd
from keras.layers import Input
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import yaml
from train import parse_config, get_features

Using TensorFlow backend.


In [2]:
from collections import namedtuple
Option = namedtuple("MyStruct", "inputFile tree config outputDir")

options = Option(
    inputFile = '../data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth.z',
    tree = 't_allpar_new',
    config = 'train_config_conv1d_small.yml',
    outputDir = '../conv1d_weights/'
)

print("Loading configuration from", options.config)
config = open(options.config, 'r')
yamlConfig =  yaml.load(config, Loader=yaml.FullLoader)
print(yamlConfig)

Loading configuration from train_config_conv1d_small.yml
{'Inputs': ['j1_ptrel', 'j1_etarot', 'j1_phirot', 'j1_erel', 'j_index'], 'Labels': ['j_g', 'j_q', 'j_w', 'j_z', 'j_t', 'j_index'], 'KerasModel': 'conv1d_small_model', 'KerasModelRetrain': 'conv1d_small_model_constraint', 'KerasLoss': 'categorical_crossentropy', 'L1Reg': 0, 'NormalizeInputs': 1, 'InputType': 'Conv1D', 'MaxParticles': 10}


In [3]:
X_train_val, X_test, y_train_val, y_test, labels  = get_features(options, yamlConfig)

(5119909,)
('j_ptfrac', 'j_pt', 'j_eta', 'j_mass', 'j_tau1_b1', 'j_tau2_b1', 'j_tau3_b1', 'j_tau1_b2', 'j_tau2_b2', 'j_tau3_b2', 'j_tau32_b1', 'j_tau32_b2', 'j_zlogz', 'j_c1_b0', 'j_c1_b1', 'j_c1_b2', 'j_c2_b1', 'j_c2_b2', 'j_d2_b1', 'j_d2_b2', 'j_d2_a1_b1', 'j_d2_a1_b2', 'j_m2_b1', 'j_m2_b2', 'j_n2_b1', 'j_n2_b2', 'j_tau1_b1_mmdt', 'j_tau2_b1_mmdt', 'j_tau3_b1_mmdt', 'j_tau1_b2_mmdt', 'j_tau2_b2_mmdt', 'j_tau3_b2_mmdt', 'j_tau32_b1_mmdt', 'j_tau32_b2_mmdt', 'j_c1_b0_mmdt', 'j_c1_b1_mmdt', 'j_c1_b2_mmdt', 'j_c2_b1_mmdt', 'j_c2_b2_mmdt', 'j_d2_b1_mmdt', 'j_d2_b2_mmdt', 'j_d2_a1_b1_mmdt', 'j_d2_a1_b2_mmdt', 'j_m2_b1_mmdt', 'j_m2_b2_mmdt', 'j_n2_b1_mmdt', 'j_n2_b2_mmdt', 'j_mass_trim', 'j_mass_mmdt', 'j_mass_prun', 'j_mass_sdb2', 'j_mass_sdm1', 'j_multiplicity', 'j1_px', 'j1_py', 'j1_pz', 'j1_e', 'j1_pdgid', 'j1_erel', 'j1_pt', 'j1_ptrel', 'j1_eta', 'j1_etarel', 'j1_etarot', 'j1_phi', 'j1_phirel', 'j1_phirot', 'j1_deltaR', 'j1_costheta', 'j1_costhetarel', 'j1_e1mcosthetarel', 'j_index', '

In [4]:
from models.models import conv1d_small_model
keras_model = conv1d_small_model(Input(shape=X_train_val.shape[1:]), y_train_val.shape[1], l1Reg=yamlConfig['L1Reg'] )

startlearningrate=0.0001
adam = Adam(lr=startlearningrate)
keras_model.compile(optimizer=adam, loss=[yamlConfig['KerasLoss']], metrics=['accuracy'])


callbacks=all_callbacks(stop_patience=1000, 
                        lr_factor=0.5,
                        lr_patience=10,
                        lr_epsilon=0.000001, 
                        lr_cooldown=2, 
                        lr_minimum=0.0000001,
                        outputDir=options.outputDir)

keras_model.fit(X_train_val, y_train_val, batch_size = 1024, epochs = 100,
                validation_split = 0.25, shuffle = True, callbacks = callbacks.callbacks)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 4)             0         
_________________________________________________________________
conv1_relu (Conv1D)          (None, 10, 3)             51        
_________________________________________________________________
conv2_relu (Conv1D)          (None, 5, 2)              26        
_________________________________________________________________
conv3_relu (Conv1D)          (None, 2, 1)              9         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2)                 0         
_________________________________________________________________
fc1_relu (Dense)             (None, 5)                 15        
____________________________________________

D:\Softwares\Anacoda\envs\tf1\lib\site-packages\keras\callbacks\callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '



Train on 14889 samples, validate on 4963 samples


Epoch 1/100
14889/14889 [==============================] - 1s 57us/step - loss: 1.6179 - accuracy: 0.6233 - val_loss: 1.6327 - val_accuracy: 0.6337

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00001: val_loss improved from inf to 1.63270, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00001: val_loss improved from inf to 1.63270, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00001: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00001: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***


Epoch 2/100
14889/14889 [==============================] - 0s 17us/step - loss: 1.6112 - accuracy: 0.6503 - val_loss: 1.6257 - val_accuracy: 0.6609

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00002: val_loss improved from 1.63270 to 1.62571, saving model to ../conv1d_weights//KERAS_check_best_


Epoch 00012: val_loss improved from 1.58086 to 1.57715, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00012: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00012: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 13/100
14889/14889 [==============================] - 0s 17us/step - loss: 1.5626 - accuracy: 0.7968 - val_loss: 1.5736 - val_accuracy: 0.7965

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00013: val_loss improved from 1.57715 to 1.57357, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00013: val_loss improved from 1.57715 to 1.57357, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00013: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00013: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 14/100
14889/14889 [==============================] 

Epoch 00025: val_loss improved from 1.53968 to 1.53681, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00025: val_loss improved from 1.53968 to 1.53681, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00025: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00025: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 26/100
14889/14889 [==============================] - 0s 17us/step - loss: 1.5248 - accuracy: 0.8414 - val_loss: 1.5340 - val_accuracy: 0.8376

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00026: val_loss improved from 1.53681 to 1.53400, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00026: val_loss improved from 1.53681 to 1.53400, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00026: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00026: saving model to ../conv1d_weight

14889/14889 [==============================] - 0s 17us/step - loss: 1.4916 - accuracy: 0.8622 - val_loss: 1.5003 - val_accuracy: 0.8567

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00038: val_loss improved from 1.50308 to 1.50028, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00038: val_loss improved from 1.50308 to 1.50028, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00038: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00038: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 39/100
14889/14889 [==============================] - 0s 17us/step - loss: 1.4888 - accuracy: 0.8639 - val_loss: 1.4975 - val_accuracy: 0.8584

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00039: val_loss improved from 1.50028 to 1.49746, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00039: val_loss improved from 1.50028 to


***callbacks end***

Epoch 51/100
14889/14889 [==============================] - 0s 18us/step - loss: 1.4547 - accuracy: 0.8767 - val_loss: 1.4631 - val_accuracy: 0.8708

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00051: val_loss improved from 1.46601 to 1.46310, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00051: val_loss improved from 1.46601 to 1.46310, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00051: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00051: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 52/100
14889/14889 [==============================] - 0s 18us/step - loss: 1.4518 - accuracy: 0.8776 - val_loss: 1.4602 - val_accuracy: 0.8723

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00052: val_loss improved from 1.46310 to 1.46019, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00052


Epoch 00063: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 64/100
14889/14889 [==============================] - 0s 17us/step - loss: 1.4169 - accuracy: 0.8870 - val_loss: 1.4249 - val_accuracy: 0.8789

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00064: val_loss improved from 1.42788 to 1.42488, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00064: val_loss improved from 1.42788 to 1.42488, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00064: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00064: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 65/100
14889/14889 [==============================] - 0s 17us/step - loss: 1.4139 - accuracy: 0.8879 - val_loss: 1.4219 - val_accuracy: 0.8799

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00065: val_loss improved from 1.42488 to

Epoch 00076: val_loss improved from 1.39181 to 1.38879, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00076: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00076: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 77/100
14889/14889 [==============================] - 0s 16us/step - loss: 1.3785 - accuracy: 0.8934 - val_loss: 1.3858 - val_accuracy: 0.8868

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00077: val_loss improved from 1.38879 to 1.38579, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00077: val_loss improved from 1.38879 to 1.38579, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00077: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00077: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 78/100
14889/14889 [==============================] -


***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00089: val_loss improved from 1.35255 to 1.34954, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00089: val_loss improved from 1.35255 to 1.34954, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00089: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00089: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 90/100
14889/14889 [==============================] - 0s 17us/step - loss: 1.3399 - accuracy: 0.8963 - val_loss: 1.3465 - val_accuracy: 0.8904

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00090: val_loss improved from 1.34954 to 1.34650, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00090: val_loss improved from 1.34954 to 1.34650, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00090: saving model to ../conv1d_weights//KERAS_ch

# Eval

In [5]:
from __future__ import print_function
import six
import sys
import os
from optparse import OptionParser
from keras.models import load_model, Model, model_from_json
from argparse import ArgumentParser
from keras import backend as K
import numpy as np
# fix random seed for reproducibility
seed = 39
np.random.seed(seed)
import h5py
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
import itertools
sys.path.append(os.getcwd() + '/..')
from models.constraints import ZeroSomeWeights
from keras.utils.generic_utils import get_custom_objects
get_custom_objects().update({"ZeroSomeWeights": ZeroSomeWeights})
import yaml
from train import parse_config, get_features
# from eval import makeRoc
from eval import plot_confusion_matrix
from layers.quantized_layers import Clip, BinaryDense, TernaryDense, QuantizedDense
from models.models import binary_tanh, ternary_tanh, quantized_relu

In [13]:
from collections import namedtuple
Option = namedtuple("MyStruct", "inputModel inputFile tree config jsonModel outputDir")

options = Option(
#     inputModel = '../weights/KERAS_conv1d_weights.h5',
    inputModel = './train_simple_conv1d/KERAS_check_best_model_weights.h5',
    inputFile = '../data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth.z',
    tree = 't_allpar_new',
#     config = 'train_config_conv1d.yml',
    config = 'train_config_conv1d_small.yml',
    jsonModel = './train_simple_conv1d/KERAS_conv1d.json',
    outputDir = 'eval_con1d_1/'
)

print("Loading configuration from", options.config)
config = open(options.config, 'r')
yamlConfig =  yaml.load(config, Loader=yaml.FullLoader)
print(yamlConfig)

Loading configuration from train_config_conv1d_small.yml
{'Inputs': ['j1_ptrel', 'j1_etarot', 'j1_phirot', 'j1_erel', 'j_index'], 'Labels': ['j_g', 'j_q', 'j_w', 'j_z', 'j_t', 'j_index'], 'KerasModel': 'conv1d_small_model', 'KerasModelRetrain': 'conv1d_small_model_constraint', 'KerasLoss': 'categorical_crossentropy', 'L1Reg': 0, 'NormalizeInputs': 1, 'InputType': 'Conv1D', 'MaxParticles': 10}


In [15]:
X_train_val, X_test, y_train_val, y_test, labels  = get_features(options, yamlConfig)

(5119909,)
('j_ptfrac', 'j_pt', 'j_eta', 'j_mass', 'j_tau1_b1', 'j_tau2_b1', 'j_tau3_b1', 'j_tau1_b2', 'j_tau2_b2', 'j_tau3_b2', 'j_tau32_b1', 'j_tau32_b2', 'j_zlogz', 'j_c1_b0', 'j_c1_b1', 'j_c1_b2', 'j_c2_b1', 'j_c2_b2', 'j_d2_b1', 'j_d2_b2', 'j_d2_a1_b1', 'j_d2_a1_b2', 'j_m2_b1', 'j_m2_b2', 'j_n2_b1', 'j_n2_b2', 'j_tau1_b1_mmdt', 'j_tau2_b1_mmdt', 'j_tau3_b1_mmdt', 'j_tau1_b2_mmdt', 'j_tau2_b2_mmdt', 'j_tau3_b2_mmdt', 'j_tau32_b1_mmdt', 'j_tau32_b2_mmdt', 'j_c1_b0_mmdt', 'j_c1_b1_mmdt', 'j_c1_b2_mmdt', 'j_c2_b1_mmdt', 'j_c2_b2_mmdt', 'j_d2_b1_mmdt', 'j_d2_b2_mmdt', 'j_d2_a1_b1_mmdt', 'j_d2_a1_b2_mmdt', 'j_m2_b1_mmdt', 'j_m2_b2_mmdt', 'j_n2_b1_mmdt', 'j_n2_b2_mmdt', 'j_mass_trim', 'j_mass_mmdt', 'j_mass_prun', 'j_mass_sdb2', 'j_mass_sdm1', 'j_multiplicity', 'j1_px', 'j1_py', 'j1_pz', 'j1_e', 'j1_pdgid', 'j1_erel', 'j1_pt', 'j1_ptrel', 'j1_eta', 'j1_etarel', 'j1_etarot', 'j1_phi', 'j1_phirel', 'j1_phirot', 'j1_deltaR', 'j1_costheta', 'j1_costhetarel', 'j1_e1mcosthetarel', 'j_index', '

In [8]:
json_file = open(options.jsonModel, 'r')
model = json_file.read()
json_file.close()
model = model_from_json(model, custom_objects={'ZeroSomeWeights':ZeroSomeWeights,
                                                           'BinaryDense': BinaryDense,
                                                           'TernaryDense': TernaryDense,
                                                           'QuantizedDense': QuantizedDense,
                                                           'binary_tanh': binary_tanh,
                                                           'ternary_tanh': ternary_tanh,
                                                           'quantized_relu': quantized_relu,
                                                           'Clip': Clip})
# load weights into new model
model.load_weights(options.inputModel)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 7)            0         
_________________________________________________________________
conv1_relu (Conv1D)          (None, 100, 32)           8992      
_________________________________________________________________
conv2_relu (Conv1D)          (None, 100, 32)           40992     
_________________________________________________________________
conv3_relu (Conv1D)          (None, 100, 32)           40992     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
_________________________________________________________________
fc1_relu (Dense)             (None, 50)                160050    
_________________________________________________________________
output_softmax (Dense)       (None, 5)                 255 

In [9]:
def makeRoc(features_val, labels, labels_val, model):
    print('in makeRoc()')
    if 'j_index' in labels: labels.remove('j_index')

    predict_test = model.predict(features_val)

    df = pd.DataFrame()
    
    fpr = {}
    tpr = {}
    auc1 = {}
    
    plt.figure()       
    for i, label in enumerate(labels):
        df[label] = labels_val[:,i]
        df[label + '_pred'] = predict_test[:,i]
        
        fpr[label], tpr[label], threshold = roc_curve(df[label],df[label+'_pred'])

        auc1[label] = auc(fpr[label], tpr[label])
            
        plt.plot(tpr[label],fpr[label],label='%s tagger, AUC = %.1f%%'%(label.replace('j_',''),auc1[label]*100.))
    plt.semilogy()
    plt.xlabel("Signal Efficiency")
    plt.ylabel("Background Efficiency")
    plt.ylim(0.001,1)
    plt.grid(True)
    plt.legend(loc='upper left')
    plt.figtext(0.25, 0.90,'hls4ml',fontweight='bold', wrap=True, horizontalalignment='right', fontsize=14)
    #plt.figtext(0.35, 0.90,'preliminary', style='italic', wrap=True, horizontalalignment='center', fontsize=14) 
    plt.savefig("mygraph.png")
#     plt.savefig('%s/ROC.pdf'%(options.outputDir))
#     plt.savefig('%s/ROC.pdf' %(outputDir))
    return predict_test

In [10]:
print("Plot ROC curve")
y_predict = makeRoc(X_test, labels, y_test, model)

Plot ROC curve
in makeRoc()


In [16]:
print("Plot ROC curve")
y_predict = makeRoc(X_test, labels, y_test, keras_model)

Plot ROC curve
in makeRoc()


In [12]:
keras_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 4)             0         
_________________________________________________________________
conv1_relu (Conv1D)          (None, 10, 3)             51        
_________________________________________________________________
conv2_relu (Conv1D)          (None, 5, 2)              26        
_________________________________________________________________
conv3_relu (Conv1D)          (None, 2, 1)              9         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2)                 0         
_________________________________________________________________
fc1_relu (Dense)             (None, 5)                 15        
_________________________________________________________________
output_softmax (Dense)       (None, 5)                 30  